In [8]:
# Import libraries
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Create model
model = gp.Model()

# Add decision variables for budget allocation per channel
num_channels = 3
num_products = 3
budget_vars = model.addVars(num_channels, lb=0.0, vtype=GRB.CONTINUOUS, name='budget')

# Define coefficients
conversion_rates = [
    [0.04, 0.01, 0.015],  # Clothing
    [0, 0.03, 0.015],  # Beauty Products
    [0.01, 0, 0.015],  # Home Decor
]

avg_ticket_size = [
    [25, 55, 55],   # Clothing
    [0, 60, 70 ],   # Beauty Products
    [40, 0,80],   # Home Decor
]

cost_per_click = [1.1, 1.6, 1.9]  # Cost per view per channel

# Set the objective function (maximize revenue)
model.setObjective(gp.quicksum((
    (avg_ticket_size[p][i] * conversion_rates[p][i] * budget_vars[i] / cost_per_click[i]) for p in range(num_products) for i in range(num_channels))) , GRB.MAXIMIZE)

# Set the total budget constraint
total_budget = 10000  # Total available budget
model.addConstr(gp.quicksum(budget_vars[i] for i in range(num_channels)) <= total_budget, name='total_budget')

# Add constraint for at least 15% budget per channel
min_budget_percent = 0.15
for i in range(num_channels):
    model.addConstr(budget_vars[i] >= min_budget_percent * total_budget, name=f'min_budget_channel_{i + 1}')

# Add constraint for total transactions per product
min_transactions_per_product = [50, 55, 60]
for p in range(num_products):  # Products
    model.addConstr(gp.quicksum(conversion_rates[p][i] * budget_vars[i] for i in range(num_channels)) >= min_transactions_per_product[p], name=f'min_conversions_product_{p + 1}')

# Add constraint for total clicks
min_clicks = 7000
model.addConstr(gp.quicksum(budget_vars[i] / cost_per_click[i] for i in range(num_channels)) >= min_clicks, name='min_clicks')

# Add constraint for maximum cost
max_cost_percent = 0.80
model.addConstr(gp.quicksum(cost_per_click[i] * budget_vars[i] for i in range(num_channels)) <= max_cost_percent * gp.quicksum(avg_ticket_size[p][i] * budget_vars[i] * conversion_rates[p][i] for p in range(num_products) for i in range(num_channels)), name='max_cost')


# Optimize the model
model.optimize()

results = {
    'Channel': [],
    'ROAS': [],
    'Budget': [],
    'Clicks': [],
    'Transactions': [],
    'Revenue': [],
    'Budget %': [],
    'Clicks %': [],
    'Transactions %': [],
    'Revenue %': []
}

total_budget_allocated = sum(budget_vars[i].x for i in range(num_channels))
total_clicks = sum(budget_vars[i].x / cost_per_click[i] for i in range(num_channels))
total_transactions = sum(conversion_rates[p][i] * budget_vars[i].x for p in range(num_products) for i in range(num_channels))
total_revenue = sum(avg_ticket_size[p][i] * conversion_rates[p][i] * budget_vars[i].x / cost_per_click[i] for p in range(num_products) for i in range(num_channels))


if model.status == GRB.OPTIMAL:
    for i in range(num_channels):
        channel_budget = budget_vars[i].x
        clicks = channel_budget / cost_per_click[i]
        transactions = sum(conversion_rates[p][i] * channel_budget for p in range(num_products))
        revenue = sum(avg_ticket_size[p][i] * conversion_rates[p][i] * channel_budget / cost_per_click[i] for p in range(num_products))
        roas = revenue/channel_budget
        
        budget_percent = (channel_budget / total_budget_allocated) * 100
        clicks_percent = (clicks / total_clicks) * 100
        transactions_percent = (transactions / total_transactions) * 100
        revenue_percent = (revenue / total_revenue) * 100
        
        results['Channel'].append(f'Channel {i + 1}')
        results['ROAS'].append(roas)
        results['Budget'].append(channel_budget)
        results['Clicks'].append(clicks)
        results['Transactions'].append(transactions)
        results['Revenue'].append(revenue)
        results['Budget %'].append(budget_percent)
        results['Clicks %'].append(clicks_percent)
        results['Transactions %'].append(transactions_percent)
        results['Revenue %'].append(revenue_percent)
else:
    print("No solution found.")

# Create a pandas DataFrame
summary_df = pd.DataFrame(results)

# Display the DataFrame
display(summary_df)


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 3 columns and 19 nonzeros
Model fingerprint: 0x8774565b
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+04]
Presolve removed 6 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5441163e+04   2.536633e+02   0.000000e+00      0s
       1    1.4524805e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.452480469e+04


,Channel,ROAS,Budget,Clicks,Transactions,Revenue,Budget %,Clicks %,Transactions %,Revenue %
0,Channel 1,1.272727,4150.78125,3773.4375,207.539062,5282.812500,41.507812,53.906250,44.800283,36.370971
1,Channel 2,1.468750,1500.00000,937.5000,60.000000,2203.125000,15.000000,13.392857,12.951861,15.168018
2,Channel 3,1.618421,4349.21875,2289.0625,195.714844,7038.867188,43.492188,32.700893,42.247856,48.461011
